In [17]:
# basic libraries
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import torchvision.transforms as transforms
from torchvision import datasets
from PIL import Image

In [2]:
# manual seed
torch.manual_seed(42)

In [28]:
# check GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [45]:
# transforms
custom_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.5, 0.5, 0.5], std = [0.5, 0.5, 0.5])
])

In [46]:
# Image paths
dataset_dir = '/kaggle/input/dog-and-cat-classification-dataset/PetImages'
dataset = datasets.ImageFolder(root = dataset_dir, transform = custom_transform)

In [47]:
# split Dataset
train_size = int(0.7 * len(dataset))
test_size = len(dataset) - train_size

In [48]:
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [49]:
# DataLoader object
train_loader = DataLoader(train_dataset, batch_size = 32, shuffle = True, pin_memory = True)
test_loader = DataLoader(test_dataset, batch_size = 32, pin_memory = True)

In [55]:
# define NN class
class MyModel(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size = 3, padding = 1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2),

            nn.Conv2d(32, 64, kernel_size = 3, padding = 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
        )
        with torch.no_grad():
            dummy_input = torch.zeros(1, *input_size)
            conv_output = self.conv(dummy_input)
            self.flattened_size = conv_output.view(1, -1).size(1)
            
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.flattened_size, 128),
            nn.ReLU(),
            nn.Dropout(p = 0.3),

            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(p = 0.3),

            nn.Linear(64, 1)
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.classifier(x)
        return x.squeeze(1)

In [56]:
# define parameters
learning_rate = 0.001
epochs = 50
input_size = (3, 128, 128)

In [57]:
# define model
model = MyModel(input_size).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [60]:
# training loop
for epoch in range(epochs):
    total_epoch_loss = 0
    for batch_features, batch_labels in train_loader:
        batch_features = batch_features.to(device)
        batch_labels = batch_labels.to(device)
        outputs = model(batch_features)
        loss = criterion(outputs, batch_labels.float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_epoch_loss += loss.item()
    avg_loss = total_epoch_loss / len(train_loader)
    print(f'Epoch: {epoch + 1}, Loss: {avg_loss}')

/usr/local/lib/python3.11/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Epoch: 1, Loss: 0.6573153453073728
Epoch: 2, Loss: 0.6228316989633675
Epoch: 3, Loss: 0.5856279327076139
Epoch: 4, Loss: 0.5567578903826761
Epoch: 5, Loss: 0.5287478379837119
Epoch: 6, Loss: 0.5127258373243935
Epoch: 7, Loss: 0.48575566689972466
Epoch: 8, Loss: 0.4635796510977109
Epoch: 9, Loss: 0.43935163139316236
Epoch: 10, Loss: 0.41698216844411395
Epoch: 11, Loss: 0.4080860091192849
Epoch: 12, Loss: 0.3898026291315037
Epoch: 13, Loss: 0.37227740505168816
Epoch: 14, Loss: 0.35734784156356475
Epoch: 15, Loss: 0.3407648527752983
Epoch: 16, Loss: 0.3317643035972794
Epoch: 17, Loss: 0.31753129277793757
Epoch: 18, Loss: 0.30262837929470665
Epoch: 19, Loss: 0.2964360183292159
Epoch: 20, Loss: 0.2823536638204533
Epoch: 21, Loss: 0.27680289787555307
Epoch: 22, Loss: 0.2653281272370266
Epoch: 23, Loss: 0.25818999456939795
Epoch: 24, Loss: 0.24730178613004564
Epoch: 25, Loss: 0.2483635695695441
Epoch: 26, Loss: 0.24082992883089255
Epoch: 27, Loss: 0.2428715773866643
Epoch: 28, Loss: 0.2363199

In [61]:
# eval mode
model.eval()

MyModel(
  (conv): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=65536, out_features=128, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
    (6): Dropout(p=0.3, inplace=False)
    (7): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [67]:
# evaluation
total = 0
correct = 0
with torch.no_grad():
    for batch_features, batch_labels in test_loader:
        batch_features = batch_features.to(device)
        batch_labels = batch_labels.to(device).float()
        outputs = model(batch_features)
        probs = torch.sigmoid(outputs)
        preds = (probs > 0.5).float()
        preds = preds.view(-1)
        batch_labels = batch_labels.view(-1)
        correct += (preds == batch_labels).sum().item()
        total += batch_labels.size(0)
acc = correct / total
print(f'Accuracy: {acc}')

Accuracy: 0.8126666666666666
